In [1]:
import pandas as pd
import numpy as np
from numpy import math
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-24 23:31:26,0.26855,0.0,0.0,0.0,2022-06-24,23:31:26,23
1,2022-06-25 00:31:26,0.26855,0.0,0.0,0.0,2022-06-25,00:31:26,0
2,2022-06-25 01:31:26,0.26855,0.0,0.0,0.0,2022-06-25,01:31:26,1
3,2022-06-25 01:32:26,0.26855,0.0,0.0,0.0,2022-06-25,01:32:26,1
4,2022-06-25 01:33:26,0.31738,0.0,0.0,0.0,2022-06-25,01:33:26,1


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
2027,2022-06-26 13:05:30,2022-06-26,13:05:30,13,14.62402,1.37439,20.099107,0.020099
2028,2022-06-26 13:06:30,2022-06-26,13:06:30,13,16.77246,5.39126,90.424693,0.090425
2029,2022-06-26 13:07:30,2022-06-26,13:07:30,13,21.67969,4.71279,102.171826,0.102172
2030,2022-06-26 13:08:31,2022-06-26,13:08:31,13,21.99707,0.00000,0.000000,0.000000
2031,2022-06-26 13:09:31,2022-06-26,13:09:31,13,15.18555,2.41965,36.743716,0.036744


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-24T00:00:00.000000000', '2022-06-25T00:00:00.000000000',
       '2022-06-26T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
length_today_hourly_values = len(today_hourly_values)
length_today_hourly_values
# today_hourly_values

14

In [7]:
filter_daily_values = df[(df['Date'] > '2022-06-24') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
1,2022-06-25,0,0.0
2,2022-06-25,1,0.0
3,2022-06-25,1,0.0
4,2022-06-25,1,0.0
5,2022-06-25,1,0.0
...,...,...,...
1239,2022-06-25,23,0.0
1240,2022-06-25,23,0.0
1241,2022-06-25,23,0.0
1242,2022-06-25,23,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values

,Date,Hour,Power (KW)
0,2022-06-25,0,0.000000
1,2022-06-25,1,0.000000
2,2022-06-25,2,0.000000
3,2022-06-25,3,0.000000
4,2022-06-25,4,0.000000
5,2022-06-25,5,0.006974
6,2022-06-25,6,0.087022
7,2022-06-25,7,0.194788
8,2022-06-25,8,0.896399
9,2022-06-25,9,1.671361


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_data.drop(['Date', 'Time', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status'], axis = 1, inplace = True)
weather_data.tail(25)

,SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
23,0.0,12.4,70,10
24,0.0,11.3,69,9
25,0.0,10.8,68,7
26,0.0,10.5,68,32
27,0.0,10.7,69,58
28,0.0,11.1,63,83
29,13.9,11.4,67,66
30,63.7,11.7,71,49
31,123.9,12.9,69,31
32,141.6,14.1,64,47


In [11]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail(25)

,Power (KW),SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
23,0.0,0.0,0.0,0,0
24,NaN,0.0,0.0,0,0
25,NaN,0.0,0.0,0,0
26,NaN,0.0,0.0,0,0
27,NaN,0.0,0.0,0,0
28,NaN,0.0,0.0,0,0
29,NaN,13.9,11.4,67,66
30,NaN,63.7,11.7,71,49
31,NaN,123.9,12.9,69,31
32,NaN,141.6,14.1,64,47


In [12]:
count_total_rows = len(df1) - 24

In [13]:
independent_columns = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:count_total_rows]

In [14]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [15]:
reg = linear_model.LinearRegression(fit_intercept = False)
reg.fit(independent_columns, dependent_column)

LinearRegression(fit_intercept=False)

#### Is model well fit or not?

In [16]:
r_sq = reg.score(independent_columns, dependent_column)
r_sq

0.8718439553088033

In [17]:
reg.intercept_

0.0

In [18]:
reg.coef_

array([ 0.00587437,  0.02953689, -0.02255836,  0.0265523 ])

In [19]:
forcasted_data = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']].tail(24)

In [20]:
return_array = list(reg.predict(forcasted_data))

In [21]:
# predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
# predicted_data

In [22]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [23]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [24]:
data_dict = {'Date': current_date_24, 'Hour': hours_24, 'Power (KW)': return_array}

In [25]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-06-26,0,0.000000
1,2022-06-26,1,0.000000
2,2022-06-26,2,0.000000
3,2022-06-26,3,0.000000
4,2022-06-26,4,0.000000
5,2022-06-26,5,0.659416
6,2022-06-26,6,0.419198
7,2022-06-26,7,0.375455
8,2022-06-26,8,1.052504
9,2022-06-26,9,1.634490


## Evaluating the model

#### MSE = mean_squared_error(actual value, predicted value)

In [26]:
mean_sq_error = metrics.mean_squared_error(today_hourly_values['Power (KW)'], 
                                           data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_sq_error

0.48002319446991576

In [27]:
root_mean_sq_error = np.sqrt(mean_sq_error)
root_mean_sq_error

0.692837061992151

In [28]:
mean_ab_error = metrics.mean_absolute_error(today_hourly_values['Power (KW)'], 
                                            data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_ab_error

0.405555017155252

#### R squared

In [29]:
r_squared = metrics.r2_score(today_hourly_values['Power (KW)'], 
                             data_dataframe['Power (KW)'].head(length_today_hourly_values))
r_squared

0.12660138870442206

In [30]:
# dependent_variable = 'Power (KW)'

In [31]:
# independent_variables = df1.columns.tolist()
# independent_variables.remove(dependent_variable)

In [32]:
# independent_variables

In [33]:
# X = df1[independent_variables].values

In [34]:
# y = df1[dependent_variable].values

In [35]:
# X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [36]:
# X_train

In [37]:
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

In [38]:
# y_pred = regressor.predict(X_test)
# y_pred

In [39]:
# math.sqrt(mean_squared_error(y_test, y_pred))

In [40]:
# r2_score(y_test, y_pred)